## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-01-07-03-59 +0000,nyt,Iran Says Supreme Leader Killed in U.S.-Israel...,https://www.nytimes.com/live/2026/02/28/world/...
1,2026-03-01-06-56-46 +0000,wapo,A celebration in the largest Iranian community...,https://www.washingtonpost.com
2,2026-03-01-06-56-46 +0000,wapo,Live updates: Iran’s supreme leader killed in ...,https://www.washingtonpost.com/world/2026/02/2...
3,2026-03-01-06-49-00 +0000,wsj,Iranian Supreme Leader Ayatollah Khamenei Dies,https://www.wsj.com/world/middle-east/iranian-...
4,2026-03-01-06-44-57 +0000,nyt,See a Breakdown of Ayatollah Khamenei and Othe...,https://www.nytimes.com/2026/02/28/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2630/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,iran,92
47,trump,42
5,strikes,37
13,attack,20
8,iranian,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
56,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...,258
2,2026-03-01-06-56-46 +0000,wapo,Live updates: Iran’s supreme leader killed in ...,https://www.washingtonpost.com/world/2026/02/2...,239
42,2026-03-01-02-19-45 +0000,bbc,Iran's Supreme Leader killed in joint US Israe...,https://www.bbc.com/news/articles/c70n9wlkx3lo...,229
47,2026-03-01-01-52-12 +0000,nypost,Multiple members of Supreme Leader Ayatollah A...,https://nypost.com/2026/02/28/world-news/multi...,223
204,2026-02-28-10-14-09 +0000,nypost,"Read, watch President Trump’s full speech afte...",https://nypost.com/2026/02/28/us-news/read-pre...,215


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
56,258,2026-03-01-01-01-13 +0000,wapo,"See where U.S., Israeli strikes have hit Iran ...",https://www.washingtonpost.com/investigations/...
220,111,2026-02-28-07-10-48 +0000,nypost,"US, Israel launch ‘Operation Epic Fury’ attack...",https://nypost.com/2026/02/28/us-news/us-israe...
47,77,2026-03-01-01-52-12 +0000,nypost,Multiple members of Supreme Leader Ayatollah A...,https://nypost.com/2026/02/28/world-news/multi...
101,43,2026-02-28-21-51-45 +0000,cbc,"Ottawa's Iranian diaspora shares hopes, fears ...",https://www.cbc.ca/news/canada/ottawa/ottawa-s...
34,41,2026-03-01-02-53-00 +0000,wsj,Democrats in Congress led the charge to releas...,https://www.wsj.com/politics/policy/epstein-fi...
164,41,2026-02-28-15-04-01 +0000,nypost,Saudi Arabia condemns Iran’s attacks on Gulf n...,https://nypost.com/2026/02/28/us-news/saudi-ar...
128,29,2026-02-28-19-13-49 +0000,latimes,"He saw an abandoned trailer. Then, he uncovere...",https://www.latimes.com/california/story/2026-...
57,28,2026-03-01-01-00-00 +0000,wsj,Inheritance is one of the last reliable ways f...,https://www.wsj.com/economy/housing/in-califor...
204,27,2026-02-28-10-14-09 +0000,nypost,"Read, watch President Trump’s full speech afte...",https://nypost.com/2026/02/28/us-news/read-pre...
187,27,2026-02-28-12-00-00 +0000,wsj,House retirements are really piling up. From E...,https://www.wsj.com/politics/policy/congress-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
